# MNist com KNN

Neste experimento foram exploradas configurações de hiper-parâmetros do algoritmo KNN, variando a quantidade de vizinhos mais próximos entre 3,5,7 e 9 e também os pesos possíveis entre 'uniform' e 'distance'. Mantive apenas a métrica de distância Euclidiana, porque a distância Manhattan demorou demais para executar e não apresentou nenhum ganho que justificasse a diferença de tempo de processamento, saltando de 20 segundos para 12 minutos. 

Para garantir uma reproducibilidade entre este experimento e os demais, o conjunto de dados foi separado em 10 pastas com a técnica de Cross Validation utilizando o estado randômico 42.

A combinação dos hiper-parâmetros foi feita utilizando Grid Search, que cruza todas as opções dos dicionários e responde qual é a melhor configuração encontrada. Estas parametrizações são utilizadas para criar um modelo final, que é serializado em um objeto Pickle.

In [7]:
from sklearn.datasets import fetch_openml

import joblib
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.neighbors import KNeighborsClassifier

import time

In [2]:
mnist = fetch_openml('mnist_784')

In [3]:
X = mnist["data"]
y = mnist["target"]

print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


O objeto $param_grid$ recebe a combinação de todos os hiper-parâmetros que pretendo comparar, como a quantidade de vizinhos próximos e o peso.

Também estou definindo a segmentação dos dados em 10 pastas e um random_state de 42 para permitir reproducibilidade ao executar este experimento novamente.

In [4]:
param_grid = [{'n_neighbors':[3,5,7,9],
               'weights':['uniform', 'distance'],
               'metric':['euclidean']}]

kfolds = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [5]:
knn = KNeighborsClassifier()

inicio = time.time()
grid_search_knn = GridSearchCV(knn, param_grid, cv=kfolds, verbose=3)
resultado_modelo = grid_search_knn.fit(X,y)
termino = time.time()
print("--- %s segundos para treinar o modelo ---" % (termino - inicio))


Fitting 10 folds for each of 8 candidates, totalling 80 fits
[CV 1/10] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.974 total time=  20.5s
[CV 2/10] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.976 total time=  20.0s
[CV 3/10] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.971 total time=  20.0s
[CV 4/10] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.973 total time=  19.9s
[CV 5/10] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.973 total time=  19.9s
[CV 6/10] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.971 total time=  19.9s
[CV 7/10] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.975 total time=  19.8s
[CV 8/10] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.976 total time=  19.9s
[CV 9/10] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.973 total time=  19.8s
[CV 10/10] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.9

Dentro do objeto $Grid\_Search\_Knn$ é possível recuperar qual foi o melhor conjunto de parâmetros, qual é o melhor *Estimator* e a pontuação que esta combinação de valores alcançou.

In [15]:
print("Melhores Parâmetros")
print(grid_search_knn.best_params_)
print("**************")
print("Melhores Estimadores")
print(grid_search_knn.best_estimator_)
print("**************")
print("Melhores Pontuações")
print(grid_search_knn.best_score_)
print("**************")

Melhores Parâmetros
{'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
**************
Melhores Estimadores
KNeighborsClassifier(metric='euclidean', n_neighbors=3, weights='distance')
**************
Melhores Pontuações
0.9744714285714284
**************


Por fim, é possível serializar toda essa configuração do treino do modelo em um objeto único. Neste caso, estou usando o Pickle que é do próprio pacote Sklearn.


In [14]:
joblib.dump(grid_search_knn, "modelo_knn_mnist.pkl")

['modelo_knn_mnist.pkl']